In [4]:
import numpy as np
import essentia.standard as estd
import essentia as ess
import scipy
import pandas as pd
import csv
import re
import os
import IPython.display as ipd
import tempfile
import random


In [5]:
# Set base directory where the dataset is located

#Songs
Songs = ['AllOfMe', 'AutumnLeaves', 'HaveYouMetMissJones', 'justFriends', 'Solar', 'TakeTheATrain', 'thereWillNeverBeAnotherYou']

#Directories
Directorie = ['Songs_v2/AllOfMe', 'Songs_v2/AutumnLeaves','Songs_v2/HaveYouMetMissJones','Songs_v2/justFriends','Songs_v2/Solar','Songs_v2/TakeTheATrain','Songs_v2/thereWillNeverBeAnotherYou'] 


In [67]:
d={}
x = 0
wavs = []
wavs = os.listdir(Directorie[6])
                    ####CHANGE HERE############################

    
#  "Song Name” _ “Segment number” _ “Slice  number” _ “Performance  Level”.  
    
for lst in wavs:
    if '.wav' in lst: 
        #Read files from the folder
        nome = re.search('(.+?)_', lst).group(1)
        segment_id = re.search('_(.+?)_', lst).group(1)
        slice_id = re.search(segment_id + '_(.+?)_', lst).group(1)
        
        pf_id = re.search(segment_id +'_'+ slice_id + '_(.+)', lst).group(1)
        pf_id = pf_id[:-4]
        
        analysis_file = lst[:-4]
        analysis_file = analysis_file + '.json'
        
                   ####CHANGE HERE################################
        location = Directorie[6] + '/' +lst
        audio = estd.MonoLoader(filename=location)()
        
        #Insert in the dictionary
        d[x] = {}
        d[x]['Name'] = nome
        d[x]['Segment Number'] = segment_id
        d[x]['slice Number'] = slice_id
        d[x]['Performance_Level'] = pf_id
        d[x]['Analysis_file'] = analysis_file
        d[x]['Audio'] = audio
        
        x = x+1


In [68]:
len(d)

19

In [69]:
def extract_features(audio_frame, pool):
    '''
    Extracts multiple features using Essentia's Extractor
    
    Arguments:
        - audio_frame:[float] - audio frame
    Returns:
        - results:pool – Essentia's pool datastructure with aggregated features
    '''
    extractor = estd.Extractor(highLevel=True, 
                    midLevel=True,
                    rhythm=True,
                    tuning=True,
                    lowLevelFrameSize=1024,
                    lowLevelHopSize = 512,
                   )
    
    pool.merge(extractor(audio_frame))
    pool.remove('rhythm.onset_times')
    pool.remove('rhythm.beats_loudness')
    pool.remove('rhythm.beats_position')
    pool.remove('rhythm.bpm_estimates')
    pool.remove('rhythm.bpm_intervals')
    pool.remove('rhythm.beats_loudness_band_ratio')
    pool.remove('tonal.chords_progression')
    pool.remove('rhythm.histogram')
    
    aggrPool = estd.PoolAggregator(defaultStats = [ 'mean', 'stdev' ])(pool)
    return aggrPool

In [70]:
count = 0

while count < len(d):
    
    audio = d[count]['Audio']
    filename = d[count]['Analysis_file']
    print(filename)
   
    pool = ess.Pool()
    pool = extract_features(audio, pool)
        
    #Append complexity feature
    
    pool.add('Performance_Level', d[count]['Performance_Level'])
        
    #write in file
    estd.YamlOutput(filename= 'Songs_v2/JSON' + '/'+ filename[:-5] + '_' + '.json', format = 'json')(pool)
        
    
    count = count + 1
      
    

thereWillNeverBeAnotherYou_1_9_low.json
thereWillNeverBeAnotherYou_2_5_mid.json
thereWillNeverBeAnotherYou_1_1_low.json
thereWillNeverBeAnotherYou_2_4_mid.json
thereWillNeverBeAnotherYou_1_8_low.json
thereWillNeverBeAnotherYou_1_3_low.json
thereWillNeverBeAnotherYou_1_2_low.json
thereWillNeverBeAnotherYou_1_14_low.json
thereWillNeverBeAnotherYou_2_2_mid.json
thereWillNeverBeAnotherYou_1_7_low.json
thereWillNeverBeAnotherYou_1_11_low.json
thereWillNeverBeAnotherYou_2_3_mid.json
thereWillNeverBeAnotherYou_1_10_low.json
thereWillNeverBeAnotherYou_1_6_low.json
thereWillNeverBeAnotherYou_2_1_mid.json
thereWillNeverBeAnotherYou_1_12_low.json
thereWillNeverBeAnotherYou_1_4_low.json
thereWillNeverBeAnotherYou_1_5_low.json
thereWillNeverBeAnotherYou_1_13_low.json
